# 3. Capstone Pre-processing Training & Data Development

In [1]:
# Libraries imported for this notebook.

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller

In [2]:
# Read Lake_Fictitious_AquaRealTime_Data_EDA into a dataframe, formatted dates, and indexed dates.

df = pd.read_excel('Lake_Fictitious_AquaRealTime_Data_EDA.xlsx', parse_dates=True, index_col=[0])

In [3]:
# Looked at shape

df.shape

(53850, 5)

In [4]:
# Printed columns in data

for col in df.columns:
    print(col)

light
waterTemp
turbidity
phycocyanin
chlorA


## Feature Engineering

### Domain-specific

In [5]:
# Added a column that is 'turb+chla'.

df['turb+chla'] = df['turbidity'] + df['chlorA']

### Temporal

#### Differencing

All metrics need to be detrended so I performed differencing to compute the differences between consecutive data points. This technique is ofter perfomed for time series models and can transform a non-stationary time series into a stationary one and remove trends and seasonality. Overall, differencing can be beneficial because it stabilizes the mean and variance of the data, making it easier to identify underlying patterns and relationships.

In [6]:
# Added first-order differenced featureds to df.

# Performed first-order differencing on all columns
first_differences = df.diff().add_prefix('1st_')

# Concatenated the results with the original DataFrame
df = pd.concat([df, first_differences], axis=1).dropna()

In [7]:
# Looked at just 'phycocyanin' to check differencing calculations.

selected_columns = ['phycocyanin', '1st_phycocyanin']
new_df = df.loc[:, selected_columns]
new_df.head(10)

,phycocyanin,1st_phycocyanin
timeString,,
2021-01-27 11:30:00,23.489999,-0.940001
2021-01-27 12:00:00,22.549999,-0.940001
2021-01-27 12:30:00,23.960000,1.410001
2021-01-27 13:00:00,25.370001,1.410001
2021-01-27 13:30:00,25.840000,0.469999
2021-01-27 14:00:00,26.309999,0.469999
2021-01-27 14:30:00,28.660000,2.350001
2021-01-27 15:00:00,31.010000,2.350001
2021-01-27 15:30:00,32.655000,1.645000


In [8]:
# Checked feature names

for column_title in df.columns:
    print(column_title)

light
waterTemp
turbidity
phycocyanin
chlorA
turb+chla
1st_light
1st_waterTemp
1st_turbidity
1st_phycocyanin
1st_chlorA
1st_turb+chla


Now I need to run the KPSS and ADF tests on the differenced metrics to see if my detrending was effective.

In [9]:
# Redefined function to run KPSS test.

def kpss_test(timeseries, column_name):
    print('Results of KPSS Test for column "{}":'.format(column_name))
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic', 'p-value', '#Lags Used'])
    for key, value in kpsstest[3].items():
        kpss_output['Critical Value (%s)' % key] = value
    print(kpss_output)
    print()

In [10]:
# Ran KPSS for first-order differenced metrics.

for column in df.columns:
    if '1st_' in column:
        kpss_test(df[column], column)

Results of KPSS Test for column "1st_light":
Test Statistic            0.000374
p-value                   0.100000
#Lags Used               32.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64

Results of KPSS Test for column "1st_waterTemp":
Test Statistic            0.069169
p-value                   0.100000
#Lags Used               92.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64

Results of KPSS Test for column "1st_turbidity":
Test Statistic             0.03403
p-value                    0.10000
#Lags Used               396.00000
Critical Value (10%)       0.34700
Critical Value (5%)        0.46300
Critical Value (2.5%)      0.57400
Critical Value (1%)        0.73900
dtype: float64

Results of KPSS Test for column "1st_phycocyanin":
Test Statistic         

/var/folders/77/lhc9l51s19q1rxd2jv1wfxf40000gn/T/ipykernel_34600/651794571.py:5: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpsstest = kpss(timeseries, regression='c', nlags="auto")
/var/folders/77/lhc9l51s19q1rxd2jv1wfxf40000gn/T/ipykernel_34600/651794571.py:5: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpsstest = kpss(timeseries, regression='c', nlags="auto")
/var/folders/77/lhc9l51s19q1rxd2jv1wfxf40000gn/T/ipykernel_34600/651794571.py:5: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpsstest = kpss(timeseries, regression='c', nlags="auto")
/var/folders/77/lhc9l51s19q1rxd2jv1wfxf40000gn/T/ipykernel_34600/651794571.p

The p-value for all metrics are >0.10 which is more than the significance level of 0.05. Therefore, I fail to reject the null hypothesis of stationarity for all metrics. They are all likely stationary according to the KPSS test.

In [11]:
# Redefined function to run ADF test.

def adf_test(timeseries, column_name):
    print('Results of ADF Test for column "{}":'.format(column_name))
    adftest = adfuller(timeseries, autolag='AIC')
    adf_output = pd.Series(adftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in adftest[4].items():
        adf_output['Critical Value (%s)' % key] = value
    print(adf_output)
    print()

In [12]:
# Ran ADF for first-order differenced metrics.

for column in df.columns:
    if '1st_' in column:
        adf_test(df[column], column)

Results of ADF Test for column "1st_light":
Test Statistic                   -55.492990
p-value                            0.000000
#Lags Used                        55.000000
Number of Observations Used    53793.000000
Critical Value (1%)               -3.430472
Critical Value (5%)               -2.861594
Critical Value (10%)              -2.566799
dtype: float64

Results of ADF Test for column "1st_waterTemp":
Test Statistic                   -33.973327
p-value                            0.000000
#Lags Used                        58.000000
Number of Observations Used    53790.000000
Critical Value (1%)               -3.430472
Critical Value (5%)               -2.861594
Critical Value (10%)              -2.566799
dtype: float64

Results of ADF Test for column "1st_turbidity":
Test Statistic                   -39.855307
p-value                            0.000000
#Lags Used                        56.000000
Number of Observations Used    53792.000000
Critical Value (1%)               -3

The p-values for all metrics are less than 0.05, indicating strong evidence against the presence of a unit root. Thus, I reject the null hypothesis for these variables, suggesting that they are likely stationary according to the ADF test.

Together, the KPSS and ADF tests indicate that all first-order differenced metrics are stationary. I do not need to perform second-order differencing.

#### Lag Features

I created lagged versions of the differenced metrics of 1 through five time periods (half-hour). This may need to be increased for modelling since blue-green algae usually takes a few days to grow after stimultion. Or not if I use models that create their own lags.

In [13]:
# Lagged differenced features by 1 through 5 time periods.

lagged_columns = []

lagged_df = pd.DataFrame()

for lag in range(1, 21):
    relevant_columns = [col for col in df.columns if '1st_' in col]
    lagged_columns.extend([f'{var}_lag_{lag}' for var in relevant_columns])
    lagged_data = {f"{var}_lag_{lag}": df[var].shift(lag) for var in relevant_columns}
    lagged_df = pd.concat([lagged_df, pd.DataFrame(lagged_data)], axis=1)

df = pd.concat([df, lagged_df], axis=1)

In [14]:
# Dropped rows containng NaN as a result of lagged features.

df.dropna(inplace=True)

In [15]:
# Looked at just phycocyanin to check lagging.

selected_columns = [col for col in df.columns if '1st_phycocyanin' in col]
new_df = df.loc[:, selected_columns]
new_df

,1st_phycocyanin,1st_phycocyanin_lag_1,1st_phycocyanin_lag_2,1st_phycocyanin_lag_3,1st_phycocyanin_lag_4,1st_phycocyanin_lag_5,1st_phycocyanin_lag_6,1st_phycocyanin_lag_7,1st_phycocyanin_lag_8,1st_phycocyanin_lag_9,...,1st_phycocyanin_lag_11,1st_phycocyanin_lag_12,1st_phycocyanin_lag_13,1st_phycocyanin_lag_14,1st_phycocyanin_lag_15,1st_phycocyanin_lag_16,1st_phycocyanin_lag_17,1st_phycocyanin_lag_18,1st_phycocyanin_lag_19,1st_phycocyanin_lag_20
timeString,,,,,,,,,,,,,,,,,,,,,
2021-01-27 21:30:00,0.235,0.235001,0.235000,-0.470000,-0.470001,-0.470000,-0.470000,-0.704999,-0.704999,-0.940001,...,1.644999,1.645000,2.350001,2.350001,0.469999,0.469999,1.410001,1.410001,-0.940001,-0.940001
2021-01-27 22:00:00,0.235,0.235000,0.235001,0.235000,-0.470000,-0.470001,-0.470000,-0.470000,-0.704999,-0.704999,...,-0.940000,1.644999,1.645000,2.350001,2.350001,0.469999,0.469999,1.410001,1.410001,-0.940001
2021-01-27 22:30:00,-0.235,0.235000,0.235000,0.235001,0.235000,-0.470000,-0.470001,-0.470000,-0.470000,-0.704999,...,-0.940001,-0.940000,1.644999,1.645000,2.350001,2.350001,0.469999,0.469999,1.410001,1.410001
2021-01-27 23:00:00,-0.235,-0.235000,0.235000,0.235000,0.235001,0.235000,-0.470000,-0.470001,-0.470000,-0.470000,...,-0.704999,-0.940001,-0.940000,1.644999,1.645000,2.350001,2.350001,0.469999,0.469999,1.410001
2021-01-27 23:30:00,0.470,-0.235000,-0.235000,0.235000,0.235000,0.235001,0.235000,-0.470000,-0.470001,-0.470000,...,-0.704999,-0.704999,-0.940001,-0.940000,1.644999,1.645000,2.350001,2.350001,0.469999,0.469999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23 05:30:00,0.000,1.220000,-3.650000,1.220000,0.000000,0.000000,0.000000,-1.220000,1.220000,0.000000,...,0.000000,0.000000,0.000000,1.220000,0.000000,-2.430000,1.210000,1.220000,0.000000,0.000000
2024-02-23 06:00:00,0.000,0.000000,1.220000,-3.650000,1.220000,0.000000,0.000000,0.000000,-1.220000,1.220000,...,1.210000,0.000000,0.000000,0.000000,1.220000,0.000000,-2.430000,1.210000,1.220000,0.000000
2024-02-23 06:30:00,1.210,0.000000,0.000000,1.220000,-3.650000,1.220000,0.000000,0.000000,0.000000,-1.220000,...,0.000000,1.210000,0.000000,0.000000,0.000000,1.220000,0.000000,-2.430000,1.210000,1.220000


In [16]:
# Added time unit features.

df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

In [17]:
# Printed list of column titles to check results of lagging.

for column_title in df.columns:
    print(column_title)

light
waterTemp
turbidity
phycocyanin
chlorA
turb+chla
1st_light
1st_waterTemp
1st_turbidity
1st_phycocyanin
1st_chlorA
1st_turb+chla
1st_light_lag_1
1st_waterTemp_lag_1
1st_turbidity_lag_1
1st_phycocyanin_lag_1
1st_chlorA_lag_1
1st_turb+chla_lag_1
1st_light_lag_2
1st_waterTemp_lag_2
1st_turbidity_lag_2
1st_phycocyanin_lag_2
1st_chlorA_lag_2
1st_turb+chla_lag_2
1st_light_lag_3
1st_waterTemp_lag_3
1st_turbidity_lag_3
1st_phycocyanin_lag_3
1st_chlorA_lag_3
1st_turb+chla_lag_3
1st_light_lag_4
1st_waterTemp_lag_4
1st_turbidity_lag_4
1st_phycocyanin_lag_4
1st_chlorA_lag_4
1st_turb+chla_lag_4
1st_light_lag_5
1st_waterTemp_lag_5
1st_turbidity_lag_5
1st_phycocyanin_lag_5
1st_chlorA_lag_5
1st_turb+chla_lag_5
1st_light_lag_6
1st_waterTemp_lag_6
1st_turbidity_lag_6
1st_phycocyanin_lag_6
1st_chlorA_lag_6
1st_turb+chla_lag_6
1st_light_lag_7
1st_waterTemp_lag_7
1st_turbidity_lag_7
1st_phycocyanin_lag_7
1st_chlorA_lag_7
1st_turb+chla_lag_7
1st_light_lag_8
1st_waterTemp_lag_8
1st_turbidity_lag_8
1st_p

# Data Save

In [18]:
# Saved dataframe.

df.to_excel('Lake_Fictitious_AquaRealTime_Data_Cleaned_Pre_Process_Train.xlsx', index=True)